In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from math import sqrt
from math import log2
from IPython.display import display
%matplotlib inline

from sklearn.metrics import r2_score
from sklearn.metrics import mean_poisson_deviance
from sklearn.metrics import mean_gamma_deviance
from sklearn.metrics import median_absolute_error
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.pipeline import make_pipeline
import time
import matplotlib as mpl

from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor



from sklearn.preprocessing import StandardScaler,PowerTransformer,MinMaxScaler,QuantileTransformer,normalize
from sklearn.model_selection import train_test_split, learning_curve, ShuffleSplit
from sklearn.feature_selection import VarianceThreshold, SelectKBest
from sklearn.model_selection import KFold
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

from sklearn.model_selection import train_test_split

import pathlib
import os
import sys

# for selection the right path
import os,sys,inspect,pathlib
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

from common.model_trainer_reg import *
from common.regression_plotfunctions import *

from GD.LinearRegression import LinearRegression
from KNN.KNNRegressor import KNNRegressor

import moneyball_preprocession


In [2]:
# Custom functions

module_path = pathlib.Path(os.getcwd()).parent
if str(module_path) not in sys.path:
    sys.path.append(str(module_path))
print(sys.path)

from common.dataset_grabber import get_data_path

['/home/alexander/Dokumente/WS2020/MaschinLearning/ML_2020/Ex2', '/home/alexander/Dokumente/WS2020/MaschinLearning/ML_2020/Ex2', '/home/alexander/Dokumente/WS2020/MaschinLearning/ML_2020/Ex2', '/home/alexander/Dokumente/WS2020/MaschinLearning/ML_2020/Ex2', '/home/alexander/Dokumente/WS2020/MaschinLearning/ML_2020/Ex2/Moneyball', '', '/home/alexander/ngsuite/ngsolve-install/lib/python3/dist-packages', '/home/alexander/ngsuite/ngsolve-install/bin', '/home/alexander/.local/bin', '/usr/local/sbin', '/usr/local/bin', '/usr/sbin', '/usr/bin', '/sbin', '/bin', '/usr/games', '/usr/local/games', '/snap/bin', '/home/alexander/.vscode/extensions/ms-toolsai.jupyter-2020.11.399280825/pythonFiles', '/home/alexander/.vscode/extensions/ms-toolsai.jupyter-2020.11.399280825/pythonFiles/lib/python', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '/home/alexander/.local/lib/python3.7/site-packages', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages

In [3]:
datapath = get_data_path("Moneyball", "baseball.csv")
display(datapath)

PosixPath('/home/alexander/Dokumente/WS2020/MaschinLearning/ML_2020/Ex2/Datasets/Moneyball/baseball.csv')

In [4]:
df_raw = pd.read_csv(datapath)

# Train and Test

In [5]:
X, Y = moneyball_preprocession.preprocessing(df_raw, drop_missing_values=False)

## SGD-Regression

In [9]:
MODEL = SGDRegressor
NAME = "SGD"
PATH = "out/"+NAME+"/"
params = {"alpha" : [0.001,0.001],
          "max_iter" : [1000,10000]}

n_splits = 2
modeltrainer = ModelTrainer(MODEL, params, X, Y, Variationerror = mean_gamma_deviance, NameOfError = "MGE", thread_cnt=1)
########### train with TrainTestSplit  ###################
modeltrainer.TTSplit(test_size = 0.5)
modeltrainer.train()
results = modeltrainer.retResults(PATH + "sklearn_TTS_SGD.csv")
display(results)
############ shuffle_Cross validation  ###################
modeltrainer.CV_shuffle_split(k = n_splits, test_size = 0.5, random_state = 42)
results = modeltrainer.retResults(PATH + "sklearn_CV_SGD.csv")
display(results)




,alpha,max_iter,train_time,inference_time,R2_score,RMSE,D,N,n_test,n_train,MGE
0,0.001,1000,0.005934,0.000392,0.876040,16.047118,4,1232,616,616,0.002726
1,0.001,10000,0.006367,0.000306,0.876574,15.977959,4,1232,616,616,0.002710
2,0.001,1000,0.006782,0.000307,0.876400,16.000429,4,1232,616,616,0.002715
3,0.001,10000,0.003631,0.000317,0.876315,16.011475,4,1232,616,616,0.002721


,alpha,max_iter,train_time,inference_time,k,R2_score,RMSE,D,N,n_test,n_train,MGE
0,0.001,1000,0.005127,0.000238,1,0.876460,15.992764,4,1232,616,616,0.002713
1,0.001,10000,0.005347,0.000259,1,0.876214,16.024614,4,1232,616,616,0.002722
2,0.001,1000,0.005672,0.000192,1,0.876315,16.011463,4,1232,616,616,0.002718
3,0.001,10000,0.002173,0.000133,1,0.876488,15.989029,4,1232,616,616,0.002716
4,0.001,1000,0.002581,0.000184,2,0.868856,15.490295,4,1232,616,616,0.002510
5,0.001,10000,0.002679,0.000180,2,0.869130,15.457825,4,1232,616,616,0.002506
6,0.001,1000,0.003615,0.000278,2,0.869148,15.455786,4,1232,616,616,0.002504
7,0.001,10000,0.003729,0.000232,2,0.869198,15.449810,4,1232,616,616,0.002503


## My SGD-Regression

In [10]:
MODEL = LinearRegression
params = {"alpha" : [0.001,0.001],
          "max_iter" : [1000,10000,100000]}
n_splits = 2
modeltrainer = ModelTrainer(MODEL, params, X, Y, Variationerror = mean_gamma_deviance, NameOfError = "MGE", thread_cnt=1)
########### train with TrainTestSplit  ###################
modeltrainer.TTSplit(test_size = 0.5)
modeltrainer.train()
results = modeltrainer.retResults(PATH + "my_TTS_SGD.csv")
display(results)
############ shuffle_Cross validation  ###################
modeltrainer.CV_shuffle_split(k = n_splits, test_size = 0.5, random_state = 42)
results = modeltrainer.retResults(PATH + "my_CV_SGD.csv")
display(results)

,alpha,max_iter,train_time,inference_time,R2_score,RMSE,D,N,n_test,n_train,MGE
0,0.001,1000,0.035682,0.000025,0.875578,16.106947,4,1232,616,616,0.002735
1,0.001,10000,0.263907,0.000040,0.875562,16.108963,4,1232,616,616,0.002736
2,0.001,100000,2.650922,0.000024,0.875562,16.108963,4,1232,616,616,0.002736
3,0.001,1000,0.024448,0.000024,0.875578,16.106947,4,1232,616,616,0.002735
4,0.001,10000,0.248890,0.000025,0.875562,16.108963,4,1232,616,616,0.002736
5,0.001,100000,2.560990,0.000024,0.875562,16.108963,4,1232,616,616,0.002736


,alpha,max_iter,train_time,inference_time,k,R2_score,RMSE,D,N,n_test,n_train,MGE
0,0.001,1000,0.029226,0.000024,1,0.875578,16.106947,4,1232,616,616,0.002735
1,0.001,10000,0.262475,0.000024,1,0.875562,16.108963,4,1232,616,616,0.002736
2,0.001,100000,2.623100,0.000024,1,0.875562,16.108963,4,1232,616,616,0.002736
3,0.001,1000,0.029461,0.000033,1,0.875578,16.106947,4,1232,616,616,0.002735
4,0.001,10000,0.288127,0.000025,1,0.875562,16.108963,4,1232,616,616,0.002736
5,0.001,100000,2.476422,0.000024,1,0.875562,16.108963,4,1232,616,616,0.002736
6,0.001,1000,0.024036,0.000022,2,0.870292,15.320653,4,1232,616,616,0.002480
7,0.001,10000,0.245291,0.000023,2,0.870294,15.320330,4,1232,616,616,0.002480
8,0.001,100000,2.660519,0.000024,2,0.870294,15.320330,4,1232,616,616,0.002480
9,0.001,1000,0.027864,0.000024,2,0.870292,15.320653,4,1232,616,616,0.002480


## KNN-Regression

In [11]:
MODEL = KNeighborsRegressor
params = {"n_neighbors": [5],
        "weights" : ["uniform"],
        "algorithm": ["brute","kd_tree","ball_tree"]}
NAME = "KNN"
PATH = "out/"+NAME+"/"

n_splits = 2
modeltrainer = ModelTrainer(MODEL, params, X, Y, Variationerror = mean_gamma_deviance, NameOfError = "MGE", thread_cnt=1)
########### train with TrainTestSplit  ###################
modeltrainer.TTSplit(test_size = 0.5)
modeltrainer.train()
results = modeltrainer.retResults(PATH + "sklearn_TTS_KNN.csv")
display(results)
############ shuffle_Cross validation  ###################
modeltrainer.CV_shuffle_split(k = n_splits, test_size = 0.5, random_state = 42)
results = modeltrainer.retResults(PATH + "sklearn_CV_KNN.csv")
display(results)

,n_neighbors,weights,algorithm,train_time,inference_time,R2_score,RMSE,D,N,n_test,n_train,MGE
0,5,uniform,brute,0.000512,0.022315,0.785048,27.826299,4,1232,616,616,0.004589
1,5,uniform,kd_tree,0.001545,0.025078,0.785048,27.826299,4,1232,616,616,0.004589
2,5,uniform,ball_tree,0.005942,0.017949,0.785048,27.826299,4,1232,616,616,0.004589


,n_neighbors,weights,algorithm,train_time,inference_time,k,R2_score,RMSE,D,N,n_test,n_train,MGE
0,5,uniform,brute,0.000415,0.028895,1,0.785048,27.826299,4,1232,616,616,0.004589
1,5,uniform,kd_tree,0.001218,0.013817,1,0.785048,27.826299,4,1232,616,616,0.004589
2,5,uniform,ball_tree,0.001405,0.011285,1,0.785048,27.826299,4,1232,616,616,0.004589
3,5,uniform,brute,0.000426,0.013025,2,0.775452,26.522792,4,1232,616,616,0.004234
4,5,uniform,kd_tree,0.001511,0.011498,2,0.775452,26.522792,4,1232,616,616,0.004234
5,5,uniform,ball_tree,0.004114,0.010960,2,0.775452,26.522792,4,1232,616,616,0.004234


## my KNN-Regression

In [16]:
MODEL = KNNRegressor
params = {"n_neighbors" : [5],
           "p": [2],
           "chunk_size": [600]}
NAME = "KNN"
PATH = "out/"+NAME+"/"

n_splits = 2
modeltrainer = ModelTrainer(MODEL, params, X, Y, Variationerror = mean_gamma_deviance, NameOfError = "MGE", thread_cnt=1)
########### train with TrainTestSplit  ###################
modeltrainer.TTSplit(test_size = 0.5)
modeltrainer.train()
results = modeltrainer.retResults(PATH + "my_TTS_KNN.csv")
display(results)
############ shuffle_Cross validation  ###################
modeltrainer.CV_shuffle_split(k = n_splits, test_size = 0.5, random_state = 42)
results = modeltrainer.retResults(PATH + "my_CV_KNN.csv")
display(results)

chunking...


,n_neighbors,p,chunk_size,train_time,inference_time,R2_score,RMSE,D,N,n_test,n_train,MGE
0,5,2,600,0.000008,0.047845,0.785048,27.826299,4,1232,616,616,0.004589


chunking...
chunking...


,n_neighbors,p,chunk_size,train_time,inference_time,k,R2_score,RMSE,D,N,n_test,n_train,MGE
0,5,2,600,0.000008,0.041459,1,0.785048,27.826299,4,1232,616,616,0.004589
1,5,2,600,0.000005,0.044052,2,0.775452,26.522792,4,1232,616,616,0.004234


## RF-Regression

In [ ]:
MODEL = RandomForestRegressor
params = {"n_estimators" : [100]}
NAME = "RF"
PATH = "out/"+NAME+"/"


n_splits = 2
modeltrainer = ModelTrainer(MODEL, params, X, Y, Variationerror = mean_gamma_deviance, NameOfError = "MGE", thread_cnt=1)
########### train with TrainTestSplit  ###################
modeltrainer.TTSplit(test_size = 0.5)
modeltrainer.train()
results = modeltrainer.retResults(PATH + "sklearn_TTS_RF.csv")
display(results)
############ shuffle_Cross validation  ###################
modeltrainer.CV_shuffle_split(k = n_splits, test_size = 0.5, random_state = 42)
results = modeltrainer.retResults(PATH + "sklearn_CV_RF.csv")
display(results)

## DT-Regression

In [ ]:
MODEL = DecisionTreeRegressor
params = {"criterion": ["mse"]}
NAME = "DT"
PATH = "out/"+NAME+"/"


modeltrainer = ModelTrainer(MODEL, params, X, Y, Variationerror = mean_gamma_deviance, NameOfError = "MGE", thread_cnt=1)
########### train with TrainTestSplit  ###################
modeltrainer.TTSplit(test_size = 0.5)
modeltrainer.train()
results = modeltrainer.retResults(PATH + "sklearn_TTS_DT.csv")
display(results)
############ shuffle_Cross validation  ###################
modeltrainer.CV_shuffle_split(k = n_splits, test_size = 0.5, random_state = 42)
results = modeltrainer.retResults(PATH + "sklearn_CV_DT.csv")
display(results)